In [ ]:
import datetime
import re
from pathlib import Path

import nbformat
import melanoma

notebook_dst_dir = Path("../")

def get_title():
    today = datetime.date.today().strftime("%Y%m%d")
    regex = re.compile("\d*$")
    min_idx = 0
    for path in notebook_dst_dir.glob(f"{today}*"):
        idx = int(regex.search(path.stem)[0])
        min_idx = max(idx + 1, min_idx)
    return f"{today}_{min_idx:02d}"

nb = nbformat.v4.new_notebook()

device = 0
batch_size = 32
width = 336
height = 224
epoch = 10
n_classes = 2
title = get_title()
transfered_weights = None

cells = []
cells += [nbformat.v4.new_markdown_cell("""
## {}
""".format(title))]

cells += [nbformat.v4.new_code_cell("""import chainer
from chainer.dataset import convert

import melanoma

title = "{}"

device = {}
batch_size = {}
width = {}
height = {}
epoch = {}
n_classes = {}
output_dir = melanoma.constants.PROJECT_ROOT / "results" / title
transfered_weights = {}

model = melanoma.models.EfficientNet()
loss_func = melanoma.models.loss.sigmoid_loss_func
optimizer = chainer.optimizers.Adam()
predictor = melanoma.predictor.Predictor(model, (width, height))

if device >= 0:
    model.to_gpu(device)

optimizer.setup(model)

train_ds, val_ds, test_ds = melanoma.dataset.DatasetBuilder(img_size=(width, height), n_classes=n_classes).build()

train_itr = chainer.iterators.MultiprocessIterator(train_ds, batch_size, repeat=True)
val_itr = chainer.iterators.MultiprocessIterator(val_ds, batch_size, repeat=False)
test_itr = chainer.iterators.MultiprocessIterator(test_ds, batch_size, repeat=False)

updater = chainer.training.StandardUpdater(
    train_itr, optimizer, converter=convert.concat_examples, loss_func=loss_func, device=device
)
evaluator = chainer.training.extensions.Evaluator(
    val_itr, model, device=device, eval_func=loss_func
)
trainer = melanoma.trainer.TrainerBuilder(updater, epoch ,evaluator, output_dir).build()
""".format(
    title, device, batch_size, width, height, epoch, n_classes, transfered_weights
))]

cells += [nbformat.v4.new_code_cell("""# run train
import cProfile

profiler = cProfile.Profile()
profiler.runcall(trainer.run)""")]

cells += [nbformat.v4.new_code_cell("""# profiling
profiler.print_stats("cumulative")
profiler.dump_stats(melanoma.constants.RESULT_DIR / title / "profile.stat")""")]

cells += [nbformat.v4.new_code_cell("""# evaluation
chainer.serializers.load_npz(melanoma.constants.RESULT_DIR / title / "snapshot_main_model_{}.npz".format(epoch), model)
if device >= 0:
    model.to_gpu()
    predictor.to_gpu()
test_itr.reset()
melanoma.evaluate.evaluate(predictor, 
                           test_itr,
                           [l.name for l in melanoma.constants.Labels],
                           output_dir / f"{title}_eval",
                           device=device,
                           )""")]

cells += [nbformat.v4.new_code_cell("""# submission
npz_footer = epoch
chainer.serializers.load_npz(melanoma.constants.RESULT_DIR / title / "snapshot_model_{}.npz".format(npz_footer), model)
if device >= 0:
    model.to_gpu()
    predictor.to_gpu()

ds = melanoma.dataset.SubmissionDataset(melanoma.dataset.DATASET_ROOT / "test.csv")
itr = chainer.iterators.MultiprocessIterator(ds, batch_size, repeat=False)
melanoma.utility.start_pdb(
lambda: melanoma.evaluate.evaluate_submission(predictor, 
                           itr,
                           output_dir / f"{title}_submission_{npz_footer}",
                           device=device,
                          )
)
""")]

nb["cells"] = cells
notebook_path = notebook_dst_dir / "{}.ipynb".format(title)
with open(notebook_path, 'w') as f:
    nbformat.write(nb, f)

print(f"generate {notebook_path}")